In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd "/content/drive/MyDrive/DDU+CE"

/content/drive/MyDrive/DDU+CE


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
plt.gray()


CE_dir = '_testset_results5/'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 100

originals = torch.load(CE_dir + f'orig_batch5_{batch_size}_X.pt')
labels = torch.load(CE_dir + f'orig_batch5_100_y.pt').to(device)
labels_exp = torch.load(CE_dir + f'orig_batch5_100_y.pt').unsqueeze(1).expand(100, 10)
targets = torch.tensor(range(10)).unsqueeze(0).expand(labels_exp.size())


print('device: ', device)

device:  cpu


<Figure size 432x288 with 0 Axes>

# Analysis

In [4]:
targets = torch.tensor(range(10)).unsqueeze(0).expand(labels_exp.size())
orig_exp = originals.expand(100,10,28,28)

GAN = {}
iter = {}
im = {}
L0 = {}
L1 = {}
mask = {}

### DeDUCE

In [5]:
method = 'DeDUCE'

coeff = 4
m = 'm6'
cl = 'cl1'
max_iter = 700
p = 'p1'

GAN[method] = torch.tensor(np.load(CE_dir + f'DeDUCE_AGAN_c{int(coeff)}{m}{cl}{p}.npy').reshape(100,10))
GAN_originals = GAN[method][labels_exp == targets].unsqueeze(1).expand(batch_size, 10)
iter[method] = torch.load(CE_dir + f'DeDUCE_iter_c{int(coeff)}{m}{cl}{p}.pt')
im[method] = torch.load(CE_dir + f'DeDUCE_arr_c{int(coeff)}{m}{cl}{p}.pt').reshape(100,10,28,28)
L0[method] = (im[method] != orig_exp).sum(dim=(2,3))
L1[method] = torch.sum((im[method] - orig_exp).abs(), dim=(2,3))
mask[method] = (targets != labels_exp) & (iter[method] < max_iter)

### JSMA

In [6]:
method = 'JSMA'

coeff = 0
max_iter = 700
p = 'p1'

GAN[method] = torch.tensor(np.load(CE_dir + f'{method}_AGAN_c{int(coeff)}{p}.npy').reshape(100,10))
iter[method] = torch.load(CE_dir + f'{method}_iter_c{int(coeff)}{p}.pt')
im[method] = torch.load(CE_dir + f'{method}_arr_c{int(coeff)}{p}.pt').reshape(100,10,28,28) #.permute(0,2,1)
L0[method] = (im[method] != orig_exp).sum(dim=(2,3))
L1[method] = torch.sum((im[method] - orig_exp).abs(), dim=(2,3))
mask[method] = (targets != labels_exp) & (iter[method] < max_iter)

### VLK

In [7]:
method = 'VLK'

coeff = 0
max_iter = float('inf')

GAN[method] = torch.tensor(np.load(CE_dir + f'{method}_AGAN_c{int(coeff)}.npy').reshape(100,10))
im[method] = torch.load(CE_dir + f'{method}_arr_c{int(coeff)}.pt').reshape(100,10,28,28)
L0[method] = (im[method] != orig_exp).sum(dim=(2,3))
L1[method] = torch.sum((im[method] - orig_exp).abs(), dim=(2,3))
mask[method] = (targets != labels_exp) & (im['VLK'].sum((2,3)) != 0) # filter out where not converged

### REVISE

In [8]:
method = 'REVISE'

coeff = 0
max_iter = 50000
lam = 0
step_size = -5

load_method = f'REVISE_l{lam}s{step_size}'

GAN[method] = torch.tensor(np.load(CE_dir + f'{load_method}_AGAN_c{int(coeff)}.npy').reshape(100,10))
iter[method] = torch.load(CE_dir + f'{load_method}_iter_c{int(coeff)}.pt')
im[method] = torch.load(CE_dir + f'{load_method}_arr_c{int(coeff)}.pt').reshape(100,10,28,28)
L0[method] = (im[method] != orig_exp).sum(dim=(2,3))
L1[method] = torch.sum((im[method] - orig_exp).abs(), dim=(2,3))
mask[method] = (targets != labels_exp) & (iter[method] < max_iter)

In [9]:
mask[method].sum()

tensor(673)

### save results summary for testset

In [ ]:
mask_all = mask['DeDUCE'] & mask['JSMA'] & mask['REVISE'] & mask['VLK'] # compare on same examples

RESULTS = torch.empty(4,4)

for i, alg in enumerate(['DeDUCE', 'JSMA', 'VLK', 'REVISE']):
    RESULTS[i,0] = GAN[alg][mask_all].mean().item()
    RESULTS[i,1] = L0[alg][mask_all].float().mean().item()
    RESULTS[i,2] = L1[alg][mask_all].mean().item()
    RESULTS[i,3] = (900 - mask[alg].sum().item()) / 9 # failure rate in percent
torch.save(RESULTS, CE_dir+'RESULTS.pt')
  

## get means and stds over five testsets

In [30]:
OVERALL = torch.empty(5,4,4)

OVERALL[0] = torch.load('./_testset_results/RESULTS.pt')
OVERALL[1] = torch.load('./_testset_results2/RESULTS.pt')
OVERALL[2] = torch.load('./_testset_results3/RESULTS.pt')
OVERALL[3] = torch.load('./_testset_results4/RESULTS.pt')
OVERALL[4] = torch.load('./_testset_results5/RESULTS.pt')

np.set_printoptions(suppress=True)

In [31]:
print(np.round(torch.mean(OVERALL, dim=0).numpy(),2))
print(np.round(torch.std(OVERALL, dim=0).numpy(),2))

[[ 22.51  21.16  10.72   0.  ]
 [ 23.9   25.59  12.63   3.09]
 [ 22.95 155.43  38.95   0.09]
 [ 20.09 752.46  53.86  26.8 ]]
[[0.72 0.46 0.1  0.  ]
 [0.41 0.65 0.27 0.28]
 [0.93 4.15 1.19 0.2 ]
 [0.88 4.98 0.58 1.03]]


In [32]:
for i in range(len(OVERALL)):
  print(np.round(OVERALL[i].numpy(),2))

[[ 22.69  21.13  10.74   0.  ]
 [ 24.15  25.76  12.81   2.67]
 [ 22.74 155.2   39.8    0.44]
 [ 20.56 747.93  54.29  27.78]]
[[ 21.55  20.72  10.69   0.  ]
 [ 23.43  24.73  12.38   3.  ]
 [ 22.84 153.54  38.39   0.  ]
 [ 19.8  753.25  53.24  26.78]]
[[ 22.29  21.74  10.76   0.  ]
 [ 23.5   26.48  12.81   3.33]
 [ 22.02 150.88  37.77   0.  ]
 [ 19.23 749.81  53.32  26.56]]
[[ 23.54  21.5   10.85   0.  ]
 [ 24.36  25.72  12.87   3.33]
 [ 24.51 162.11  40.57   0.  ]
 [ 21.37 760.69  53.89  27.67]]
[[ 22.5   20.7   10.57   0.  ]
 [ 24.07  25.27  12.29   3.11]
 [ 22.66 155.4   38.2    0.  ]
 [ 19.47 750.62  54.56  25.22]]


In [36]:
def r2mean(x):
    return np.round(x.mean().item(), 2)
def r2std(x):
    return np.round(x.std().item(), 2)

mask_all = mask['DeDUCE'] & mask['JSMA'] & mask['REVISE'] & mask['VLK'] # compare on same examples

print('originals GAN:', r2mean(GAN['DeDUCE'][labels_exp == targets])) # 18.05 17.78 13.25

for alg in GAN.keys():
  ALIGN = '   ' if 'JSMA' in alg else ('    ' if 'VLK' in alg else ' ')
  print(alg, ALIGN, '   GAN: ', r2mean(GAN[alg][mask_all]), '  L0: ', r2mean(L0[alg][mask_all].float()), '  L1: ', r2mean(L1[alg][mask_all]), '  good:', mask[alg].sum().item())
  
# STDs
print('STDs:')
for alg in GAN.keys():
  print(alg, '    GAN: ', r2std(GAN[alg][mask_all]), '  L0: ', r2std(L0[alg][mask_all].float()), '  L1: ', r2std(L1[alg][mask_all])) 

originals GAN: 14.04
DeDUCE      GAN:  22.5   L0:  20.7   L1:  10.57   good: 900
JSMA        GAN:  24.07   L0:  25.27   L1:  12.29   good: 872
VLK         GAN:  22.66   L0:  155.4   L1:  38.2   good: 900
REVISE      GAN:  19.47   L0:  750.62   L1:  54.56   good: 673
STDs:
DeDUCE     GAN:  8.46   L0:  11.7   L1:  5.62
JSMA     GAN:  9.55   L0:  14.57   L1:  6.45
VLK     GAN:  10.52   L0:  75.5   L1:  20.02
REVISE     GAN:  7.84   L0:  113.41   L1:  12.96


In [ ]:
## REVISE reconstructions
print('REVISEd  AGAN: ', r2mean(GAN['REVISE'][targets == labels_exp])) 
rev_gans =  [14.78, 14.51, 14.26, 14.77, 14.9]

In [ ]:
print('originals AGAN:', np.mean([18.05, 17.78, 13.25, 14.09, 14.04]), np.std([18.05, 17.78, 13.25, 14.09, 14.04]))
print('REVISEd originals AGAN:', np.mean(rev_gans), np.std(rev_gans))
